In [49]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import random
import seaborn as sns
import matplotlib.pyplot as plt

### Load Data

- Concat DataFrames

In [4]:
df_list = []
for file in os.listdir("customer_data"):
    if file.endswith(".csv"):
        df = pd.read_csv(
            f"customer_data/{file}", sep=";", parse_dates=True, index_col="timestamp"
        )
        df_list.append(df)
df = pd.concat(df_list, sort=True)

- Set distinct ID for every customer

In [5]:
df.sort_index(inplace=True)
df["time"] = df.index.time
df["day"] = df.index.day_name()
df["customer_id"] = df["day"] + "_" + df["customer_no"].astype(str)


### Adding Section Order

In [6]:
enter_datetimes = df.reset_index().groupby("customer_id")["timestamp"].min()
for customer in df["customer_id"].unique():
    df.loc[
        (df.customer_id == customer) & (df.index == enter_datetimes[customer]),
        "section_order",
    ] = "first"
    
df.loc[df["location"] == "checkout", "section_order"] = "checkout"
df["section_order"].fillna("following", inplace=True)

In [419]:
df[df['customer_id'] == 'Friday_6']

,customer_no,location,time,day,customer_id,section_order
timestamp,,,,,,
2019-09-06 07:02:00,6,spices,07:02:00,Friday,Friday_6,first
2019-09-06 07:03:00,6,dairy,07:03:00,Friday,Friday_6,following
2019-09-06 07:05:00,6,drinks,07:05:00,Friday,Friday_6,following
2019-09-06 07:08:00,6,spices,07:08:00,Friday,Friday_6,following
2019-09-06 07:09:00,6,fruit,07:09:00,Friday,Friday_6,following
2019-09-06 07:10:00,6,checkout,07:10:00,Friday,Friday_6,checkout


In [420]:
df.shape

(24877, 6)

In [421]:
df["section_order"].value_counts()

following    10015
first         7445
checkout      7417
Name: section_order, dtype: int64

### Checking last state as checkout or not ? & Deleting the customers that have invalid state cycles 

In [8]:
last_values = df.groupby('customer_id')["location"].last()
last_values=last_values.reset_index()
df_customerid_no=last_values[last_values["location"]!="checkout"]
list_of_customerid_not=df_customerid_no["customer_id"].tolist()
df=df[~df['customer_id'].isin(list_of_customerid_not)]

In [9]:
df.shape

(24798, 6)

In [11]:
df_sorted = df.sort_values(["customer_id", "timestamp"])
df_sorted.head()

,customer_no,location,time,day,customer_id,section_order
timestamp,,,,,,
2019-09-06 07:00:00,1,dairy,07:00:00,Friday,Friday_1,first
2019-09-06 07:04:00,1,spices,07:04:00,Friday,Friday_1,following
2019-09-06 07:05:00,1,checkout,07:05:00,Friday,Friday_1,checkout
2019-09-06 07:06:00,10,fruit,07:06:00,Friday,Friday_10,first
2019-09-06 07:11:00,10,checkout,07:11:00,Friday,Friday_10,checkout


### Transition Matrix

In [12]:
df_sorted["next_location"] = df_sorted["location"].shift(-1)

In [13]:
df_sorted.head()

,customer_no,location,time,day,customer_id,section_order,next_location
timestamp,,,,,,,
2019-09-06 07:00:00,1,dairy,07:00:00,Friday,Friday_1,first,spices
2019-09-06 07:04:00,1,spices,07:04:00,Friday,Friday_1,following,checkout
2019-09-06 07:05:00,1,checkout,07:05:00,Friday,Friday_1,checkout,fruit
2019-09-06 07:06:00,10,fruit,07:06:00,Friday,Friday_10,first,checkout
2019-09-06 07:11:00,10,checkout,07:11:00,Friday,Friday_10,checkout,fruit


In [14]:
trans_matrix = pd.crosstab(
    df_sorted["location"], df_sorted["next_location"], normalize=0
)

In [15]:
trans_matrix.loc["checkout"][0]=1
trans_matrix.loc["checkout"][1:5]=0
trans_matrix

next_location,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.392389,0.000000,0.222318,0.189852,0.195442
drinks,0.538956,0.027256,0.000000,0.217794,0.215994
fruit,0.500784,0.236966,0.136417,0.000000,0.125833
spices,0.251672,0.323616,0.272800,0.151912,0.000000


In [16]:
trans_matrix.to_csv("data/trans_matrix.csv")

- Draw transition probabilities

In [28]:
"""
import pygraphviz as pgv
locations = ["dairy", "spices", "drinks", "fruit", "checkout"]
G = pgv.AGraph(directed=True)
for start_location in locations[:-1]:
    for end_location in locations:
        G.add_edge(
            start_location,
            end_location,
            label=np.round(P.loc[start_location, end_location], 2),
        )

G.draw("transition.png", prog="dot")
"""

'\nimport pygraphviz as pgv\nlocations = ["dairy", "spices", "drinks", "fruit", "checkout"]\nG = pgv.AGraph(directed=True)\nfor start_location in locations[:-1]:\n    for end_location in locations:\n        G.add_edge(\n            start_location,\n            end_location,\n            label=np.round(P.loc[start_location, end_location], 2),\n        )\n\nG.draw("transition.png", prog="dot")\n'

- Test Transition Matrix on CSV File

In [18]:
trans_file=pd.read_csv("data/trans_matrix.csv", index_col=0)
trans_file

,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.392389,0.000000,0.222318,0.189852,0.195442
drinks,0.538956,0.027256,0.000000,0.217794,0.215994
fruit,0.500784,0.236966,0.136417,0.000000,0.125833
spices,0.251672,0.323616,0.272800,0.151912,0.000000


In [19]:
def random_next_state(trans_matrix, current_state, states):
    return random.choices(states, weights=trans_matrix.loc[current_state,:])[0]
    

In [20]:
# declare possible states in a list and initiate current state 
states = ['checkout','dairy','drinks','fruit','spices']
current_state = 'dairy'

In [23]:
count=0
for i in range(10000):
    next_state=random_next_state(trans_file, current_state, states)
    if(next_state=="spices"):
        count+=1
print(f"""The total caunt: {count}""")  

The total caunt: 2034


# EDA

In [26]:
df.head()

,customer_no,location,time,day,customer_id,section_order
timestamp,,,,,,
2019-09-02 07:03:00,1,dairy,07:03:00,Monday,Monday_1,first
2019-09-02 07:03:00,2,dairy,07:03:00,Monday,Monday_2,first
2019-09-02 07:04:00,3,dairy,07:04:00,Monday,Monday_3,first
2019-09-02 07:04:00,4,dairy,07:04:00,Monday,Monday_4,first
2019-09-02 07:04:00,5,spices,07:04:00,Monday,Monday_5,first


- Calculate the total number of customers in each section

In [37]:
df_1=df.groupby("location")[["customer_id"]].count().sort_values("customer_id", ascending=False)
df_1

,customer_id
location,
checkout,7417
fruit,5102
dairy,4651
drinks,3889
spices,3739


In [38]:
# 1b-daily average of customers per section
df_1b=round(
    df.groupby("location")[["customer_id"]]
    .count()
    .sort_values("customer_id", ascending=False)
    / 5,
    0,
).astype(int)
df_1b
    


,customer_id
location,
checkout,1483
fruit,1020
dairy,930
drinks,778
spices,748


- Calculate the total number of customers in each section over time

In [54]:
# average number of customers per minute and location
avg_customers_section = (
    df.groupby([df.time, df.location])[["customer_id"]].count() / 5
)
avg_customers_section

customer_id
time     location             
07:00:00 dairy             0.6
         drinks            0.6
         fruit             0.8
07:01:00 checkout          0.4
         dairy             0.8
...                        ...
21:48:00 dairy             0.4
         fruit             0.2
         spices            0.4
21:49:00 checkout          1.2
21:50:00 checkout          0.4

[4364 rows x 1 columns]

- Display the number of customers at checkout over time

In [41]:
# average number of customers at checkout per minute
avg_checkouts = df[df.location == "checkout"]
avg_checkouts.groupby([avg_checkouts.time, avg_checkouts.location])[["location"]].count() / 5

,,location
time,location,
07:01:00,checkout,0.4
07:02:00,checkout,1.0
07:03:00,checkout,0.2
07:04:00,checkout,0.4
07:05:00,checkout,1.4
...,...,...
21:46:00,checkout,1.0
21:47:00,checkout,1.0
21:48:00,checkout,1.4


- Calculate the time each customer spent in the market

In [42]:
# Enter datetimes
enter_datetime = (
    df.reset_index()[["timestamp", "customer_id"]].groupby("customer_id").min()
)
enter_datetime.rename(columns={"timestamp": "enter_datetime"}, inplace=True)
enter_datetime.sort_values("enter_datetime")

# Exit datetimes
exit_datetime = (
    df.reset_index()[["timestamp", "customer_id"]].groupby("customer_id").max()
)
exit_datetime.rename(columns={"timestamp": "exit_datetime"}, inplace=True)
exit_datetime.sort_values("exit_datetime")
# Customer time spends
customer_time_spent = pd.concat([enter_datetime, exit_datetime], axis=1)
customer_time_spent["time_spent"] = (
    customer_time_spent["exit_datetime"] - customer_time_spent["enter_datetime"]
)
customer_time_spent.sort_values("enter_datetime")

,enter_datetime,exit_datetime,time_spent
customer_id,,,
Monday_1,2019-09-02 07:03:00,2019-09-02 07:05:00,0 days 00:02:00
Monday_2,2019-09-02 07:03:00,2019-09-02 07:06:00,0 days 00:03:00
Monday_4,2019-09-02 07:04:00,2019-09-02 07:08:00,0 days 00:04:00
Monday_7,2019-09-02 07:04:00,2019-09-02 07:13:00,0 days 00:09:00
Monday_3,2019-09-02 07:04:00,2019-09-02 07:06:00,0 days 00:02:00
...,...,...,...
Friday_1501,2019-09-06 21:46:00,2019-09-06 21:48:00,0 days 00:02:00
Friday_1502,2019-09-06 21:46:00,2019-09-06 21:47:00,0 days 00:01:00
Friday_1504,2019-09-06 21:47:00,2019-09-06 21:48:00,0 days 00:01:00


- Calculate the total number of customers present in the supermarket over time

In [47]:
# average number of customers present per minute
df.groupby("time")[["customer_id"]].count()/5

,customer_id
time,
07:00:00,2.0
07:01:00,1.8
07:02:00,1.8
07:03:00,3.2
07:04:00,2.8
...,...
21:46:00,2.2
21:47:00,1.6
21:48:00,2.4
